# Denison CS181/DA210 SW Lab #12 - Step 3

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules

In [ ]:
import pandas as pd
import os
import os.path
import json
import sqlalchemy as sa

#### Set credentials

In [ ]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [ ]:
def buildConnectionString(source="sqlite_book"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

---

## Part E: Table Creation

Recall the syntax for SQL `CREATE TABLE` statements:

CREATE TABLE [IF NOT EXISTS] _table-name_ ( \
&nbsp;&nbsp;&nbsp;&nbsp;_field-name_ _data-type_ _constraints_ \
&nbsp;&nbsp;&nbsp;&nbsp;[, _field-name_ _data-type_ _constraints_ ]* \
&nbsp;&nbsp;&nbsp;&nbsp;[, _table-constraint_ ]* \
)

#### **Before going further, create a new database!**

In SQLiteStudio, navigate to `Database` -> `Add a database`.  Click the green round "+" button to create a new local DB file (store it in your repository's `dbfiles` directory, along with `book` and `school`).  Name it `entertainment`.

Let's connect to our new empty `entertainment` database and add a new table:

In [ ]:
# Build the conection string
cstring = buildConnectionString("sqlite_entertainment")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

In [ ]:
# Write the create-table SQL statement
statement = """
CREATE TABLE IF NOT EXISTS movies (
    movieid INT NOT NULL,
    title VARCHAR(64) NOT NULL,
    release DATE NULL,
    rating FLOAT DEFAULT 0.0,
    PRIMARY KEY (movieid)
)
"""

# Execute the statement
try:
    connection.execute(statement)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of movies failed:", str(err))

In [ ]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

In SQLiteStudio, double-click on the `entertainment` database, and you should now see a single table, `movies`.

> You've reached the second checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 2: There is no `NULL` or `NOT NULL` constraint given for the `rating` field in the SQL statement above.  Inspect the `movies` table in SQLiteStudio.  What is the default when neither variant of this field constraint is specified?

---

# Part F: Table Population

The SQL syntax to populate data into an existing table is straightforward:

INSERT INTO _table-name_ [ ( _column-list_ ) ] \
&nbsp;&nbsp;&nbsp;&nbsp;_field-name_ VALUES ( _value-list_ )

We focus on populating one record at a time.  Note that we can use this syntax in one of two ways, depending on whether we are inserting all defined fields, or only an explicit set of fields.

In [ ]:
# Insert all fields -- try this in SQLiteStudio!
stmt = """
    INSERT INTO movies
    VALUES (109,445, 'Frozen', '2013-11-27', 7.3)
"""

In [ ]:
# Insert only some fields -- try this in SQLiteStudio!
stmt = """
    INSERT INTO movies (title, movieid)
    VALUES ('Guardians of the Galaxy', 118340)
"""

If you tried out the above commands, you can delete the rows (or not) in the table using buttons above the data in the `Data` tab in SQLiteSTudio before continuing.

We can hard-code such insert statements, but this will become tedious quickly.  Instead, we can use `sqlalchemy` to iterate through the rows in a `pandas` `DataFrame` and bind the variables for each field, inserting records one at a time.

In [ ]:
# Re-connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

In [ ]:
# The movie data we'll insert
DoL = {
    "movieid": [109445, 118340, 299536, 301518, 420818, 424694],
    "title": ["Frozen", "Guardians of the Galaxy", "Avengers: Infinity War",
              "Toy Story 4", "The Lion King", "Bohemian Rhapsody"],
    "release": ["2013-11-27", "2014-08-01", "2018-04-27",
                "2019-06-21", "2019-07-19", "2018-11-02"],
    "rating": [7.3, 7.9, 8.3, 7.6, 7.1, 8.0]
}

df = pd.DataFrame(DoL)
df

In [ ]:
# We can access the values in the row-Series
for rowlabel, rowseries in df.iterrows():
    print(str(rowlabel) + ":", rowseries["movieid"], rowseries["title"])

In [ ]:
# Prepare a SQL statement that will use variable binding
# (note that for the binding to work, we'll need to use
#  the names of the columns in our DataFrame)
stmt = """
    INSERT INTO movies
    VALUES (:movieid, :title, :release, :rating)
"""
prepare_insert = sa.sql.text(stmt)

# Iterate through the rows in the DataFrame
for rowlabel, rowseries in df.iterrows():
    # Bind variables (match :x in statement)
    bound_insert = prepare_insert.bindparams(**rowseries) # some fancy Python

    # Execute
    rp = connection.execute(bound_insert)

Note that the ** prefix on the dictionary `rowseries`, above, converts the dictionary to a set of named parameters in the function call to `bindparams`.  It's not really necessary for you to understand how this works.

Let's see how we did by querying our `movies` table.

In [ ]:
df2 = pd.read_sql_table("movies", connection)
df2

In [ ]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

---

## Part G: Try it Yourself!


**Q5:** Create a table, `television`, (programmatically in Python or via a SQL statement executed in SQLiteStudio) to represent the following table of data.  Make sure to use fields that make sense, and define a primary key.  Think about which fields may be allowed to be missing.

ID | Title    | Service | Episodes
:--|:--------|:---------|:-------------------------
0  | Stranger Things   | Netflix | 25
1  | The Crown | Netflix | 40
2  | Star Trek: Discovery   | Paramount+ | 55
3  | Star Trek: Lower Decks | Paramount+ | 20
4  | The Handmaid's Tale | Hulu | 46

In [ ]:
# Re-connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

# YOUR CODE HERE
raise NotImplementedError() # or comment out if doing this is SQLiteStudio

**Q6:** Now that you have your data, write a SQL statement to insert one record.  You should hard-code your solution for this question.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Testing cell
df = pd.read_sql_table("television", connection)
assert df.shape == (1,4)

**Q7:** Build a `pandas DataFrame`, named `tv_df`, for the remaining 4 rows.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Display the DataFrame
tv_df

In [ ]:
# Testing cell
assert tv_df.shape == (4,4)
assert list(tv_df["service"]) == ["Netflix", "Paramount+", "Paramount+", "Hulu"]
assert list(tv_df["episodes"]) == [40, 55, 20, 46]

**Q8:** Use your dataframe from the previous question to insert the remaining four rows into the `television` table.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Testing cell
df = pd.read_sql_table("television", connection)
assert df.shape == (5,4)

assert df.episodes.max() == 55
assert df.episodes.min() == 20

query = """
    SELECT title
    FROM television
    WHERE episodes = 40
"""
df2 = pd.read_sql_query(query, connection)
assert df2.shape == (1,1)
assert df2.iloc[0,0] == "The Crown"

In [ ]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

> You've reached the third (and final) checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 3: What, if anything, would have to change in your solution to Q8 if the order of the columns in the `television` table were different?

---

---

## Part H

How much time (in minutes/hours) did you spend on this lab outside of class?

YOUR ANSWER HERE